# work on the audio DataFrame

In [1]:
import numpy as np
import os
import pandas as pd
import librosa
import scipy
from scipy.stats import skew

In [2]:
from sklearn import preprocessing

In [3]:
train = pd.read_csv('trainaudio.csv')

In [4]:
train.head()

,file,label
0,00176480_nohash_0.wav,bed
1,004ae714_nohash_0.wav,bed
2,004ae714_nohash_1.wav,bed
3,00f0204f_nohash_0.wav,bed
4,00f0204f_nohash_1.wav,bed


In [5]:
def get_mfcc(name, path):
    y, sr = librosa.load(path + name, sr = 22050)
    
    # Let's make and display a mel-scaled power (energy-squared) spectrogram
    S = librosa.feature.melspectrogram(y, sr=sr, n_mels=128)
    
    # Convert to log scale (dB). We'll use the peak power (max) as reference.
    log_S = librosa.power_to_db(S, ref=np.max)
    
    # Next, we'll extract the top 13 Mel-frequency cepstral coefficients (MFCCs)
    mfcc = librosa.feature.mfcc(S=log_S, n_mfcc=13)

# Let's pad on the first and second deltas while we're at it
    delta_mfcc  = librosa.feature.delta(mfcc)
    delta2_mfcc = librosa.feature.delta(mfcc, order=2)
    
    M = np.vstack([mfcc, delta_mfcc, delta2_mfcc])
    
    return M


def transform_mfcc(features):
    return pd.Series(np.hstack((np.mean(features, axis=1),
                                np.std(features, axis=1),
                                skew(features, axis = 1),
                                np.max(features, axis = 1),
                                np.median(features, axis = 1), 
                                np.min(features, axis = 1))))

# convert the word 'bed' audios to mfcc DataFrame

In [12]:
subsetDataFrame = train[train['label'] == 'bed']

In [13]:
subsetDataFrame

,file,label
0,00176480_nohash_0.wav,bed
1,004ae714_nohash_0.wav,bed
2,004ae714_nohash_1.wav,bed
3,00f0204f_nohash_0.wav,bed
4,00f0204f_nohash_1.wav,bed
5,012c8314_nohash_0.wav,bed
6,012c8314_nohash_1.wav,bed
7,0132a06d_nohash_0.wav,bed
8,0135f3f2_nohash_0.wav,bed
9,0137b3f4_nohash_0.wav,bed


In [14]:
train_data = pd.DataFrame()
train_data['fname'] = subsetDataFrame['file']
train_data = train_data['fname'].apply(get_mfcc, path='audio_samples/train/audio/bed/').apply(transform_mfcc)
print('done loading train mfcc')

C:\Users\hafsa\Anaconda3\lib\site-packages\scipy\signal\_arraytools.py:45: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  b = a[a_slice]


done loading train mfcc


In [15]:
train_data['word'] = train['label']

In [16]:
train_data.head()

,0,1,2,3,4,5,6,7,8,9,...,225,226,227,228,229,230,231,232,233,word
0,-768.943360,75.281516,-6.127648,28.258123,7.912945,-5.839312,-9.590038,-8.889294,-0.758801,0.839811,...,-3.091953,-1.951284,-2.212016,-3.850204,-0.896324,-1.413393,-2.015955,-1.729380,-1.423088,bed
1,-587.099245,55.608292,-69.100249,81.233755,-24.713254,10.341991,-2.582527,6.359282,2.309489,6.749812,...,-4.052969,-3.102639,-2.814981,-1.955563,-3.003451,-2.092617,-1.216518,-2.623285,-1.606822,bed
2,-515.146338,68.505114,-81.329167,88.987145,-30.714232,16.078043,-5.122750,10.798063,1.443282,10.467298,...,-2.450296,-3.956337,-3.592738,-1.790476,-1.969065,-2.555002,-0.842128,-1.925176,-1.686241,bed
3,-616.001177,121.430008,-19.593220,44.188299,-5.241854,10.341423,-19.207170,16.033597,4.118385,-2.983866,...,-4.546568,-3.937329,-5.026003,-1.698162,-1.344925,-2.185441,-1.586013,-1.163988,-1.590450,bed
4,-632.850599,119.791629,-15.088769,41.959895,-4.890035,9.337283,-14.600092,15.055539,6.877917,0.945635,...,-3.424481,-3.448728,-4.094031,-2.201777,-1.859824,-1.498480,-1.479055,-1.477093,-1.547634,bed


In [17]:
train_data.to_csv('bed.csv', index = False)

# convert the word 'bird' audios to mfcc DataFrame


In [18]:
subsetDataFrame2 = train[train['label'] == 'bird']

In [ ]:
subsetDataFrame2

,file,label
1713,00b01445_nohash_0.wav,bird
1714,00f0204f_nohash_0.wav,bird
1715,00f0204f_nohash_1.wav,bird
1716,00f0204f_nohash_2.wav,bird
1717,00f0204f_nohash_3.wav,bird
1718,012c8314_nohash_0.wav,bird
1719,0132a06d_nohash_0.wav,bird
1720,0135f3f2_nohash_0.wav,bird
1721,0137b3f4_nohash_0.wav,bird
1722,014f9f65_nohash_0.wav,bird


In [ ]:
train_data2 = pd.DataFrame()
train_data2['fname'] = subsetDataFrame2['file']
train_data2 = train_data2['fname'].apply(get_mfcc, path='audio_samples/train/audio/bird/').apply(transform_mfcc)
print('done loading train mfcc')

C:\Users\hafsa\Anaconda3\lib\site-packages\scipy\signal\_arraytools.py:45: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  b = a[a_slice]


In [ ]:
train_data2['word'] = subsetDataFrame2['label']

In [ ]:
train_data2.head()

In [ ]:
train_data2.to_csv('bird.csv', index = False)

# convert the word 'cat' audios to mfcc DataFrame


In [ ]:
subsetDataFrame3 = train[train['label'] == 'cat']

In [ ]:
subsetDataFrame3

In [ ]:
train_data3 = pd.DataFrame()
train_data3['fname'] = subsetDataFrame3['file']
train_data3 = train_data3['fname'].apply(get_mfcc, path='audio_samples/train/audio/cat/').apply(transform_mfcc)
print('done loading train mfcc')

In [ ]:
train_data3['word'] = subsetDataFrame3['label']

In [ ]:
train_data3.head()

# convert the word 'happy' audios to mfcc DataFrame

In [ ]:
subsetDataFrame4 = train[train['label'] == 'happy']
train_data4 = pd.DataFrame()
train_data4['fname'] = subsetDataFrame4['file']
train_data4 = train_data4['fname'].apply(get_mfcc, path='audio_samples/train/audio/happy/').apply(transform_mfcc)
print('done loading train mfcc')

In [ ]:
train_data4.head()

In [ ]:
train_data4['word'] = subsetDataFrame4['label']

In [ ]:
train_data4.head()

# convert the word 'house' audios to mfcc DataFrame

In [ ]:
subsetDataFrame5 = train[train['label'] == 'house']
train_data5 = pd.DataFrame()
train_data5['fname'] = subsetDataFrame5['file']
train_data5 = train_data5['fname'].apply(get_mfcc, path='audio_samples/train/audio/house/').apply(transform_mfcc)
print('done loading train mfcc')

In [ ]:
train_data5.head()

In [ ]:
train_data5['word'] = subsetDataFrame5['label']

In [ ]:
train_data5.head()

In [ ]:
final_df=pd.concat([train_data,train_data2,train_data3,train_data4,train_data5])

In [ ]:
final_df.head()

In [ ]:
final_df.tail()

In [ ]:
final_df.to_csv('trainmfcc.csv', index = False)

In [ ]:
subsetDataFrame6 = train[train['label'] == 'dog']
train_data6 = pd.DataFrame()
train_data6['fname'] = subsetDataFrame6['file']
train_data6 = train_data6['fname'].apply(get_mfcc, path='audio_samples/train/audio/dog/').apply(transform_mfcc)
print('done loading train mfcc')

In [ ]:
train_data6['word'] = subsetDataFrame6['label']

In [ ]:
subsetDataFrame7 = train[train['label'] == 'down']
train_data7 = pd.DataFrame()
train_data7['fname'] = subsetDataFrame7['file']
train_data7 = train_data7['fname'].apply(get_mfcc, path='audio_samples/train/audio/down/').apply(transform_mfcc)
print('done loading train mfcc')

In [ ]:
train_data7['word'] = subsetDataFrame7['label']

In [ ]:
subsetDataFrame8 = train[train['label'] == 'eight']
train_data8 = pd.DataFrame()
train_data8['fname'] = subsetDataFrame8['file']
train_data8 = train_data8['fname'].apply(get_mfcc, path='audio_samples/train/audio/eight/').apply(transform_mfcc)
print('done loading train mfcc')

In [ ]:
train_data8['word'] = subsetDataFrame8['label']

In [ ]:
# five 

In [ ]:
subsetDataFrame9 = train[train['label'] == 'five']
train_data9 = pd.DataFrame()
train_data9['fname'] = subsetDataFrame9['file']
train_data9 = train_data9['fname'].apply(get_mfcc, path='audio_samples/train/audio/five/').apply(transform_mfcc)
print('done loading train mfcc')

In [ ]:
train_data9['word'] = subsetDataFrame9['label']

In [ ]:
# four 

In [ ]:
subsetDataFrame10 = train[train['label'] == 'four']
train_data10 = pd.DataFrame()
train_data10['fname'] = subsetDataFrame10['file']
train_data10 = train_data10['fname'].apply(get_mfcc, path='audio_samples/train/audio/four/').apply(transform_mfcc)
print('done loading train mfcc')

In [ ]:
train_data10['word'] = subsetDataFrame10['label']

In [ ]:
# go

In [ ]:
subsetDataFrame11 = train[train['label'] == 'go']
train_data11 = pd.DataFrame()
train_data11['fname'] = subsetDataFrame11['file']
train_data11 = train_data11['fname'].apply(get_mfcc, path='audio_samples/train/audio/go/').apply(transform_mfcc)
print('done loading train mfcc')

In [ ]:
train_data11['word'] = subsetDataFrame11['label']

In [ ]:
# left 

In [ ]:
subsetDataFrame12 = train[train['label'] == 'left']
train_data12 = pd.DataFrame()
train_data12['fname'] = subsetDataFrame12['file']
train_data12 = train_data12['fname'].apply(get_mfcc, path='audio_samples/train/audio/left/').apply(transform_mfcc)
print('done loading train mfcc')

In [ ]:
train_data12['word'] = subsetDataFrame12['label']

In [ ]:
# marvin

In [ ]:
subsetDataFrame13 = train[train['label'] == 'marvin']
train_data13 = pd.DataFrame()
train_data13['fname'] = subsetDataFrame13['file']
train_data13 = train_data13['fname'].apply(get_mfcc, path='audio_samples/train/audio/marvin/').apply(transform_mfcc)
print('done loading train mfcc')

In [ ]:
train_data13['word'] = subsetDataFrame13['label']

In [ ]:
# nine

In [ ]:
subsetDataFrame14 = train[train['label'] == 'nine']
train_data14 = pd.DataFrame()
train_data14['fname'] = subsetDataFrame14['file']
train_data14 = train_data14['fname'].apply(get_mfcc, path='audio_samples/train/audio/nine/').apply(transform_mfcc)
print('done loading train mfcc')

In [ ]:
train_data14['word'] = subsetDataFrame14['label']

In [ ]:
# no

In [ ]:
subsetDataFrame15 = train[train['label'] == 'no']
train_data15 = pd.DataFrame()
train_data15['fname'] = subsetDataFrame15['file']
train_data15 = train_data15['fname'].apply(get_mfcc, path='audio_samples/train/audio/no/').apply(transform_mfcc)
print('done loading train mfcc')

In [ ]:
train_data15['word'] = subsetDataFrame15['label']

In [ ]:
# off 

In [ ]:
subsetDataFrame16 = train[train['label'] == 'off']
train_data16 = pd.DataFrame()
train_data16['fname'] = subsetDataFrame16['file']
train_data16 = train_data16['fname'].apply(get_mfcc, path='audio_samples/train/audio/off/').apply(transform_mfcc)
print('done loading train mfcc')

In [ ]:
train_data16['word'] = subsetDataFrame16['label']

In [ ]:
# on

In [ ]:
subsetDataFrame17 = train[train['label'] == 'on']
train_data17 = pd.DataFrame()
train_data17['fname'] = subsetDataFrame17['file']
train_data17 = train_data17['fname'].apply(get_mfcc, path='audio_samples/train/audio/on/').apply(transform_mfcc)
print('done loading train mfcc')

In [ ]:
train_data17['word'] = subsetDataFrame17['label']

In [ ]:
# one 

In [ ]:
subsetDataFrame18 = train[train['label'] == 'one']
train_data18 = pd.DataFrame()
train_data18['fname'] = subsetDataFrame18['file']
train_data18 = train_data18['fname'].apply(get_mfcc, path='audio_samples/train/audio/one/').apply(transform_mfcc)
print('done loading train mfcc')

In [ ]:
train_data18['word'] = subsetDataFrame18['label']

In [ ]:
# right 

In [ ]:
subsetDataFrame19 = train[train['label'] == 'right']
train_data19 = pd.DataFrame()
train_data19['fname'] = subsetDataFrame19['file']
train_data19 = train_data19['fname'].apply(get_mfcc, path='audio_samples/train/audio/right/').apply(transform_mfcc)
print('done loading train mfcc')

In [ ]:
train_data19['word'] = subsetDataFrame19['label']

In [ ]:
# seven

In [ ]:
subsetDataFrame20 = train[train['label'] == 'seven']
train_data20 = pd.DataFrame()
train_data20['fname'] = subsetDataFrame20['file']
train_data20 = train_data20['fname'].apply(get_mfcc, path='audio_samples/train/audio/seven/').apply(transform_mfcc)
print('done loading train mfcc')

In [ ]:
train_data20['word'] = subsetDataFrame20['label']

In [ ]:
# sheila

In [ ]:
subsetDataFrame21 = train[train['label'] == 'sheila']
train_data21 = pd.DataFrame()
train_data21['fname'] = subsetDataFrame21['file']
train_data21 = train_data21['fname'].apply(get_mfcc, path='audio_samples/train/audio/sheila/').apply(transform_mfcc)
print('done loading train mfcc')

In [ ]:
train_data21['word'] = subsetDataFrame21['label']

In [ ]:
# six 

In [ ]:
subsetDataFrame22 = train[train['label'] == 'six']
train_data22 = pd.DataFrame()
train_data22['fname'] = subsetDataFrame22['file']
train_data22 = train_data22['fname'].apply(get_mfcc, path='audio_samples/train/audio/six/').apply(transform_mfcc)
print('done loading train mfcc')

In [ ]:
train_data22['word'] = subsetDataFrame22['label']

In [ ]:
# stop

In [ ]:
subsetDataFrame23 = train[train['label'] == 'stop']
train_data23 = pd.DataFrame()
train_data23['fname'] = subsetDataFrame23['file']
train_data23 = train_data23['fname'].apply(get_mfcc, path='audio_samples/train/audio/stop/').apply(transform_mfcc)
print('done loading train mfcc')

In [ ]:
train_data23['word'] = subsetDataFrame23['label']

In [ ]:
# three

In [ ]:
subsetDataFrame24 = train[train['label'] == 'three']
train_data24 = pd.DataFrame()
train_data24['fname'] = subsetDataFrame24['file']
train_data24 = train_data24['fname'].apply(get_mfcc, path='audio_samples/train/audio/three/').apply(transform_mfcc)
print('done loading train mfcc')

In [ ]:
train_data24['word'] = subsetDataFrame24['label']

In [ ]:
# tree

In [ ]:
subsetDataFrame25 = train[train['label'] == 'tree']
train_data25 = pd.DataFrame()
train_data25['fname'] = subsetDataFrame25['file']
train_data25 = train_data25['fname'].apply(get_mfcc, path='audio_samples/train/audio/tree/').apply(transform_mfcc)
print('done loading train mfcc')

In [ ]:
train_data25['word'] = subsetDataFrame25['label']

In [ ]:
# two 

In [ ]:
subsetDataFrame26 = train[train['label'] == 'two']
train_data26 = pd.DataFrame()
train_data26['fname'] = subsetDataFrame26['file']
train_data26 = train_data26['fname'].apply(get_mfcc, path='audio_samples/train/audio/two/').apply(transform_mfcc)
print('done loading train mfcc')

In [ ]:
train_data26['word'] = subsetDataFrame26['label']

In [ ]:
# up 

In [ ]:
subsetDataFrame27 = train[train['label'] == 'up']
train_data27 = pd.DataFrame()
train_data27['fname'] = subsetDataFrame27['file']
train_data27 = train_data27['fname'].apply(get_mfcc, path='audio_samples/train/audio/up/').apply(transform_mfcc)
print('done loading train mfcc')

In [ ]:
train_data27['word'] = subsetDataFrame27['label']

In [ ]:
# wow 

In [ ]:
subsetDataFrame28 = train[train['label'] == 'wow']
train_data28 = pd.DataFrame()
train_data28['fname'] = subsetDataFrame28['file']
train_data28 = train_data28['fname'].apply(get_mfcc, path='audio_samples/train/audio/wow/').apply(transform_mfcc)
print('done loading train mfcc')

In [ ]:
train_data28['word'] = subsetDataFrame28['label']

In [ ]:
# yes

In [ ]:
subsetDataFrame29 = train[train['label'] == 'yes']
train_data29 = pd.DataFrame()
train_data29['fname'] = subsetDataFrame29['file']
train_data29 = train_data29['fname'].apply(get_mfcc, path='audio_samples/train/audio/yes/').apply(transform_mfcc)
print('done loading train mfcc')

In [ ]:
train_data29['word'] = subsetDataFrame29['label']

In [ ]:
# zero 

In [ ]:
subsetDataFrame30 = train[train['label'] == 'zero']
train_data30 = pd.DataFrame()
train_data30['fname'] = subsetDataFrame30['file']
train_data30 = train_data30['fname'].apply(get_mfcc, path='audio_samples/train/audio/zero/').apply(transform_mfcc)
print('done loading train mfcc')

In [ ]:
train_data30['word'] = subsetDataFrame30['label']

In [ ]:
final_df=pd.concat([train_data,train_data2,train_data3,train_data4,train_data5,train_data6,train_data7,train_data8,train_data9,train_data10,train_data11,train_data12,train_data13,train_data14,train_data15,train_data16,train_data17,train_data18,train_data19,train_data20,train_data21,train_data22,train_data23,train_data24,train_data25,train_data26,train_data27,train_data28,train_data29,train_data30])

In [ ]:
final_df.to_csv('trainmfcc.csv', index = False)

In [ ]:
final_df.tail()

In [13]:
subsetDataFrame31 = train[train['label'] == '_background_noise_']

In [14]:
subsetDataFrame31

,file,label


In [15]:
subsetDataFrame31 = train[train['label'] == 'zero']
train_data31 = pd.DataFrame()
train_data31['fname'] = subsetDataFrame31['file']
train_data31 = train_data31['fname'].apply(get_mfcc, path='audio_samples/train/audio/_background_noise_/').apply(transform_mfcc)
print('done loading train mfcc')

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\hafsa\\Desktop\\PFA Files\\dtw-master\\audio_samples\\train\\audio\\_background_noise_\\004ae714_nohash_0.wav'